# Using the Python API

In this interactive Jupyter notebook, you will learn how to build a simple random forest site-of-metabolism (SOM) classifier using a synthetic dataset.  We also show how to use our utility for computing the FAME score, which is a similarity-based applicability domain score.

The resulting classifier shown here will be very similar to the classifier created in the [](CommandLine.ipynb) tutorial, but crucially you can use Python scripting and scikit-learn to modify the model to your liking, add new features, or use different model architectures.

## Preparing the data

In [ ]:
from ast import literal_eval

from rdkit.Chem import Draw
from rdkit.Chem.rdmolfiles import SDMolSupplier

Let's start by loading the synthetic metabolism dataset. Known SOM indices stored as SDF/MOL2 molecule properties are also extracted here.

In [ ]:
molecules = [
    mol for mol in SDMolSupplier("data/metatrans_autoannotated_cleaned/train.sdf")
]
soms = [literal_eval(mol.GetProp("soms")) for mol in molecules]

We can take a quick look at some of the molecules from the dataset, with known SOMs being highlighted in red.

In [ ]:
Draw.MolsToGridImage(molecules[:16], highlightAtomLists=soms, molsPerRow=4)

Now, we extract the individual atoms, along with their SOM label.

In [ ]:
atoms_and_labels = [
    (atom, atom.GetIdx() in soms)
    for mol, soms in zip(molecules, soms)
    for atom in mol.GetAtoms()
]

atoms, labels = zip(*atoms_and_labels)

### RDKit helper function

In [ ]:
from rdkit.Chem.rdchem import Atom, Mol
from rdkit.Chem.rdmolfiles import MolToSmiles

In order to get our atoms into a format that `FAME3R` understands, we define a small helper function.
The function converts an RDKit atom into a SMILES string that uses atom mapping information to mark that exact atom.

In [ ]:
def atom_to_marked_smiles(atom: Atom) -> str:
    mol = Mol(atom.GetOwningMol())
    mol.GetAtomWithIdx(atom.GetIdx()).SetAtomMapNum(1)

    return MolToSmiles(mol)

## Training the model

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline

from fame3r import FAME3RVectorizer

We can use the scikit-learn [`Pipeline`](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html) utility to create a simple random forest model and train it on our synthetic dataset.
The hyperparameters chosen are the same as the defaults used in the `fame3r` CLI tool.

In [ ]:
model = make_pipeline(
    FAME3RVectorizer(input="smiles", radius=5),
    RandomForestClassifier(
        n_estimators=250,
        max_depth=None,
        min_samples_split=2,
        min_samples_leaf=1,
        max_features="sqrt",
        class_weight="balanced_subsample",
        n_jobs=-1,
    ),
)

In [ ]:
model.fit([[atom_to_marked_smiles(atom)] for atom in atoms], labels)

## Making predictions

In [ ]:
from rdkit.Chem import Draw
from rdkit.Chem.rdmolfiles import MolFromSmiles

We can now use the trained model to make predictions about individiual atoms. 

For example, consider two atoms in morphine. 
First, the oxygen of the hydroxyl group attached to C3, and second the quarternary carbon C12.

In [ ]:
morphine = MolFromSmiles("CN1CC[C@]23C4=C5C=CC(O)=C4O[C@H]2[C@@H](O)C=C[C@H]3[C@H]1C5")

In [ ]:
Draw.MolToImage(morphine, highlightAtoms=[10, 4])

The model predicts that the considered hydroxyl group would be much more reactive than the quarternary carbon.
This matches basic chemical intuition, as well as the known metabolism pathway of opiates.

In [ ]:
model.predict_proba(
    [
        [atom_to_marked_smiles(morphine.GetAtomWithIdx(10))],
        [atom_to_marked_smiles(morphine.GetAtomWithIdx(4))],
    ],
)[:, 1]

## Using the FAME score

In [ ]:
from rdkit.Chem import Draw
from rdkit.Chem.rdmolfiles import MolFromSmiles
from sklearn.pipeline import make_pipeline

from fame3r import FAME3RScoreEstimator, FAME3RVectorizer

The FAME score is a similarity-based applicability domain score. We train a model for calculating that score based on atoms in the dataset in much the same way as our classification model. Crucially, we only use fingerprint descriptors here, as the FAME score is well-defined only for binary features.

In [ ]:
applicability_domain = make_pipeline(
    FAME3RVectorizer(input="smiles", radius=5, output=["fingerprint"]),
    FAME3RScoreEstimator(n_neighbors=3),
)

In [ ]:
applicability_domain.fit([[atom_to_marked_smiles(atom)] for atom in atoms], labels)

As an example, we can consider modafinil. The molecule contains a sulfoxide group, a relatively rare structural feature, as well as an amide, which is very commonly found in our dataset.

In [ ]:
modafinil = MolFromSmiles("O=S(C(c1ccccc1)c2ccccc2)CC(=O)N")

In [ ]:
Draw.MolToImage(modafinil, highlightAtoms=[1, 18])

As expected, the sulfur atom of the rarer sulfoxide group receives a much lower FAME score than the nitrogen of the more common amide.

In [ ]:
applicability_domain.predict(
    [
        [atom_to_marked_smiles(modafinil.GetAtomWithIdx(1))],
        [atom_to_marked_smiles(modafinil.GetAtomWithIdx(18))],
    ],
)